In [1]:
## This program generates CSV files containing only one type of attack along with benign traffic.
##
## Each file includes all attack instances and a randomly selected portion of benign traffic.
## The distribution ratio is set as follows:
## - Attack: 30%
## - Benign: 70%

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import random
import os
import pandas as pd
import time
seconds = time.time()
%matplotlib inline

In [4]:
base_path = "/content/drive/MyDrive/Intrusion Detection System/"

In [5]:
#this function creates a folder in the appropriate directory.
def folder(f_name):
    folder_path = os.path.join(base_path, f_name)  # Create full path

    try:
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Folder created: {folder_path}")
        else:
            print(f"Folder already exists: {folder_path}")
    except OSError:
        print("The folder could not be created!")

In [6]:
# Headers of column
main_labels=["Flow ID","Source IP","Source Port","Destination IP","Destination Port","Protocol","Timestamp","Flow Duration","Total Fwd Packets",
   "Total Backward Packets","Total Length of Fwd Packets","Total Length of Bwd Packets","Fwd Packet Length Max","Fwd Packet Length Min",
   "Fwd Packet Length Mean","Fwd Packet Length Std","Bwd Packet Length Max","Bwd Packet Length Min","Bwd Packet Length Mean","Bwd Packet Length Std",
   "Flow Bytes/s","Flow Packets/s","Flow IAT Mean","Flow IAT Std","Flow IAT Max","Flow IAT Min","Fwd IAT Total","Fwd IAT Mean","Fwd IAT Std","Fwd IAT Max",
   "Fwd IAT Min","Bwd IAT Total","Bwd IAT Mean","Bwd IAT Std","Bwd IAT Max","Bwd IAT Min","Fwd PSH Flags","Bwd PSH Flags","Fwd URG Flags","Bwd URG Flags",
   "Fwd Header Length","Bwd Header Length","Fwd Packets/s","Bwd Packets/s","Min Packet Length","Max Packet Length","Packet Length Mean","Packet Length Std",
   "Packet Length Variance","FIN Flag Count","SYN Flag Count","RST Flag Count","PSH Flag Count","ACK Flag Count","URG Flag Count","CWE Flag Count",
   "ECE Flag Count","Down/Up Ratio","Average Packet Size","Avg Fwd Segment Size","Avg Bwd Segment Size","Fwd Avg Bytes/Bulk",
   "Fwd Avg Packets/Bulk","Fwd Avg Bulk Rate","Bwd Avg Bytes/Bulk","Bwd Avg Packets/Bulk","Bwd Avg Bulk Rate","Subflow Fwd Packets","Subflow Fwd Bytes",
   "Subflow Bwd Packets","Subflow Bwd Bytes","Init_Win_bytes_forward","Init_Win_bytes_backward","act_data_pkt_fwd",
   "min_seg_size_forward","Active Mean","Active Std","Active Max","Active Min","Idle Mean","Idle Std","Idle Max","Idle Min","Label","External IP"]
main_labels=( ",".join( i for i in main_labels ) )

In [7]:
# Attack types
attacks=["BENIGN", "Bot", "DDoS", "DoS GoldenEye", "DoS Hulk", "DoS Slowhttptest", "DoS slowloris", "FTP-Patator", "Heartbleed", "Infiltration", "PortScan", "SSH-Patator", "Web Attack – Brute Force", "Web Attack – Sql Injection", "Web Attack – XSS"]

# Create attacks folder
folder("attacks")

Folder created: /content/drive/MyDrive/Intrusion Detection System/attacks


In [8]:
benign=2359289

dict_attack={
"Bot":1966,
"DDoS":41835,
"DoS GoldenEye":10293,
"DoS Hulk":231073,
"DoS Slowhttptest":5499,
"DoS slowloris":5796,
"FTP-Patator":7938,
"Heartbleed":11,
"Infiltration":36,
"PortScan":158930,
"SSH-Patator":5897,
"Web Attack - Brute Force":1507,
"Web Attack - XSS":652,
"Web Attack - Sql Injection":21}

In [9]:
# in this section, a file is opened for each attack type and is recorded at a random benign flow.
for attack in dict_attack:
    a, b = 0, 0
    attack_file_path = os.path.join(base_path, "attacks", f"{attack}.csv")

    with open(attack_file_path, "w") as ths:
        ths.write(main_labels + "\n")

        benign_num = int(benign / (dict_attack[attack] * (7 / 3)))

        all_data_path = os.path.join(base_path, "all_data.csv")
        with open(all_data_path, "r") as file:
            while True:
                try:
                    line = file.readline().strip()
                    if not line:
                        break

                    k = line.split(",")

                    if k[83] == "BENIGN":
                        rnd = random.randint(1, benign_num)
                        if rnd == 1:
                            ths.write(line + "\n")
                            b += 1
                    elif k[83] == attack:
                        ths.write(line + "\n")
                        a += 1
                except Exception as e:
                    print(f"Error processing {attack}: {e}")
                    break

    print(f"{attack} file completed\n Attack: {a}\n Benign: {b}\n\n")



Bot file completed
 Attack: 1966
 Benign: 4643


DDoS file completed
 Attack: 41835
 Benign: 98313


DoS GoldenEye file completed
 Attack: 10293
 Benign: 24065


DoS Hulk file completed
 Attack: 231073
 Benign: 589767


DoS Slowhttptest file completed
 Attack: 5499
 Benign: 13042


DoS slowloris file completed
 Attack: 5796
 Benign: 13401


FTP-Patator file completed
 Attack: 7938
 Benign: 18569


Heartbleed file completed
 Attack: 11
 Benign: 27


Infiltration file completed
 Attack: 36
 Benign: 80


PortScan file completed
 Attack: 158930
 Benign: 393197


SSH-Patator file completed
 Attack: 5897
 Benign: 13794


Web Attack - Brute Force file completed
 Attack: 1507
 Benign: 3543


Web Attack - XSS file completed
 Attack: 652
 Benign: 1548


Web Attack - Sql Injection file completed
 Attack: 21
 Benign: 46




In [10]:
#All web attack files are merged into a single file.
web_attacks = ["Web Attack - Brute Force", "Web Attack - XSS", "Web Attack - Sql Injection"]
web_attack_file = os.path.join(base_path, "attacks", "Web Attack.csv")

flag = True
for attack in web_attacks:
    attack_file_path = os.path.join(base_path, "attacks", f"{attack}.csv")

    if os.path.exists(attack_file_path):
        df = pd.read_csv(attack_file_path)

        if flag:
            df.to_csv(web_attack_file, index=False)
            flag = False
        else:
            df.to_csv(web_attack_file, index=False, header=False, mode="a")

        os.remove(attack_file_path)  # Delete individual web attack files

In [11]:
print(f"Operation completed in {time.time() - seconds} seconds")

Operation completed in 285.55239152908325 seconds
